# Homework 3: Image Convolution and Approximation Homework

**Due**: Wednesday, December 6, 9:14am, via
[Moodle](https://moodle.uni-heidelberg.de/mod/assign/view.php?id=988858)

## Group Members

- Maximilian Burr
- Christian ...

## Regulations:

We ask you to include your solutions (only the .ipynb) in a zip folder
and upload it on Moodle. Include the names of all members (not more than
3) on top of your jupyter notebook as a markdown cell. The submission
deadline is always the next week before the beginning of the class on
Wednesday at 09:14 but this time you have two weeks to work on this
homework. For clarification on the code, please try to include comments
in order for it to be understandable.

  



### Loading required packages packages

In [1]:
from skimage import io, img_as_float, img_as_ubyte
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

### Exact Adder Function

The ExactAdder function performs binary addition on three input bits (a,
b, and c). It outputs the sum (s) and carry-out (c_out) based on the
combination of input bits. The function encapsulates the logic of a full
adder, handling all possible input cases through conditional statements.
It is a fundamental component in digital circuit design.

In [2]:
def ExactAdder(a, b, c):
    if a==0 and b==0 and c==0:
        s=0
        c_out=0
    elif a==0 and b==0 and c==1:
        s=1
        c_out=0
    elif a==0 and b==1 and c==0:
        s=1
        c_out=0
    elif a==0 and b==1 and c==1:
        s=0
        c_out=1
    elif a==1 and b==0 and c==0:
        s=1
        c_out=0
    elif a==1 and b==0 and c==1:
        s=0
        c_out=1
    elif a==1 and b==1 and c==0:
        s=0
        c_out=1
    elif a==1 and b==1 and c==1:
        s=1
        c_out=1
    return s, c_out

### Approximate Adder Function<a href="#Approximate-Adder-Function" class="anchor-link">¶</a>

The ApproxAdder function attempts binary addition but deviates from
conventional logic in its handling of input combinations. In particular,
it provides an inaccurate sum (s) for the cases (0, 0, 0) and (1, 1, 1).

Failure Cases:

#### Input (0, 0, 0):<a href="#Input-(0,-0,-0):" class="anchor-link">¶</a>

Incorrect sum (s) = 1 Correct carry-out (c_out) = 0

#### Input (1, 1, 1):<a href="#Input-(1,-1,-1):" class="anchor-link">¶</a>

Incorrect sum (s) = 0 Correct carry-out (c_out) = 1

#### Considerations:<a href="#Considerations:" class="anchor-link">¶</a>

The function's approach to binary addition may have specific intentions,
but users should be cautious about its accuracy, especially in scenarios
requiring precise binary addition.

#### Function Usage:<a href="#Function-Usage:" class="anchor-link">¶</a>

Call the function with three binary inputs (a, b, and c) to obtain the
potentially inaccurate sum (s) and correct carry-out (c_out).

In [3]:
def ApproxAdder(a, b, c):
    if a==0 and b==0 and c==0:
        s=1
        c_out=0
    elif a==0 and b==0 and c==1:
        s=1
        c_out=0
    elif a==0 and b==1 and c==0:
        s=1
        c_out=0
    elif a==0 and b==1 and c==1:
        s=0
        c_out=1
    elif a==1 and b==0 and c==0:
        s=1
        c_out=0
    elif a==1 and b==0 and c==1:
        s=0
        c_out=1
    elif a==1 and b==1 and c==0:
        s=0
        c_out=1
    elif a==1 and b==1 and c==1:
        s=0
        c_out=1
    return s, c_out

### N-bit Adder Function

The `MyNbitAdder` function is designed to perform N-bit addition, where
the lower 4 bits utilize an approximate adder (`ApproxAdder`), and the
remaining bits employ an exact adder (`ExactAdder`).

#### Function Overview

1.  **Input Conversion:**

    -   Convert input integers `a` and `b` to binary and remove the
        binary prefix.

2.  **Byte Reversal:**

    -   Reverse the order of bits in both binary representations.

3.  **Equalizing Byte Length:**

    -   Ensure both binary representations have equal length by padding
        with zeros.

4.  **Bitwise Addition:**

    -   Perform bitwise addition, using either the approximate adder or
        exact adder based on the specified threshold (`approx_until`).

5.  **Result Calculation:**

    -   Calculate the total sum by summing up the bitwise results and
        accounting for any carry-over.

This function allows users to perform N-bit addition with a customizable
threshold (`approx_until`) to switch between the approximate and exact
adders. Adjust the threshold as needed for specific requirements.

In [4]:
#In 12 bit adder, lower 4 bits are implemented with approximate adder and rest of the with with exact adder
def MyNbitAdder(a,b,approx_until = 4):
    #convert to binary and cut off the first two indices (they dont belong to the number but indicate that it is binary)
    a_bin, b_bin = bin(a)[2:] , bin(b)[2:]
    
    #reverse order of bytes for the adder
    rev_a , rev_b = list(a_bin[::-1]), list(b_bin[::-1])
    
    
    #We want to make the to bytes to equalt length such that we can add 
    #--> add zeros to the shortest list until it is the same as the longest
    rev_a = rev_a + max(0, len(rev_b)-len(rev_a)) * [0]
    rev_b = rev_b + max(0, len(rev_a)-len(rev_b)) * [0]
    
    
    carry_over  = 0
    total_sum   = 0
    
    #############################################
    # approx_until = 4 #change this if u want to approximate the first bits by an approximate adder
    #############################################

    #we want to do a bitwise addition
    for index, (bit1, bit2) in enumerate( zip(rev_a, rev_b) ):

        if index < approx_until:
            #use approx_adder
            sum_element, carry_over = ApproxAdder(int(bit1), int(bit2), int(carry_over) ) 
        else:
            #use exact_adder
            sum_element, carry_over = ExactAdder(int(bit1), int(bit2), int(carry_over) )
            
        total_sum += pow(2,index)*sum_element

    total_sum += pow(2,index+1)*carry_over
    return total_sum

MyNbitAdder(3,3)

4

### Image Adder Function

The `MyAdder` function is designed to add two images represented as
arrays (`f` and `g`). It utilizes a custom N-bit adder (`MyNbitAdder`)
for element-wise addition of corresponding pixel values.

#### Function Overview<a href="#Function-Overview" class="anchor-link">¶</a>

1.  **Input Conversion:**

    -   Convert input images `f` and `g` to NumPy arrays and ensure they
        are of integer type.

2.  **Image Size Validation:**

    -   Ensure that the size of both images is the same.

3.  **Pixel-wise Addition:**

    -   Iterate through each pixel of the images and use the custom
        N-bit adder (`MyNbitAdder`) to add corresponding pixel values.

4.  **Resultant Image:**

    -   Return the resultant image as a NumPy array.

This function allows users to add two images pixel-wise using the custom
N-bit adder. Ensure that the images have the same size for proper
element-wise addition. Adjust the input images as needed for specific
use cases.

In [5]:
# Function that addes two images
def MyAdder(f,g):
    
    f=np.array(f).astype(int)
    g=np.array(g).astype(int)
    #ensure that the size of the image is the same
    
    res = np.zeros(f.shape)
    
    for i in range(f.shape[0]):
        for j in range(f.shape[1]):
            
            #we will use a custom adding function
            res[i, j] = MyNbitAdder(f[i,j],g[i,j])
            # FIXME: Why divide by 2?!
    
    return res

f = np.eye(3)
g = np.eye(3)
MyAdder(f,g)

array([[2., 1., 1.],
       [1., 2., 1.],
       [1., 1., 2.]])

### Filter Definition

The kernel or filter for convolution is defined as follows: This 3x3
filter is often used in image processing, particularly in Gaussian blur
operations. Each element in the filter contributes to the weighted sum
of pixel values during convolution, emphasizing the central pixel.
Adjustments to the filter values can impact the characteristics of the
convolution operation, such as blurring or edge detection.

In [6]:
########## Gaussian kernel/filter ##############
fltr = [
    [1, 2, 1],
    [2, 4, 2],
    [1, 2, 1]
]
fltr = np.array(fltr)

### Additional Filters

In addition to the Gaussian filter mentioned above, we provide two types
of filters classified as follows:

#### 1) Low Pass Filter

-   **f1:** Low pass average filter
-   **f2:** Low pass Gaussian filter with dimension 3x3
-   **f3:** Low pass Gaussian filter with dimension 5x5

These filters are designed for tasks such as removing noise from images
or smoothing/blurring images. The key characteristic is that the sum of
all the kernel elements should be one.

#### 2) High Pass Filter

-   **f4:** High pass horizontal vertical filter
-   **f5:** High pass diagonal filter
-   **f6:** Sobel horizontal edge filter

High pass filters are utilized for edge detection or sharpening of
images, and they have the property that the sum of all the kernel
elements should be zero.


In [32]:
filter= []

f1 = [
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 1]
]
f1 = np.array(f1)
#f1 = 1/np.sum(f1) *np.array(f1)
filter.append(f1)

#again a normalized gaussian filter
f2 = [
    [1, 2, 1],
    [2, 4, 2],
    [1, 2, 1]
] 
#f2 = 1/np.sum(f2) *np.array(f2)
f2 = np.array(f2)
filter.append(f2)

f3 = [
    [1, 4,  7,  4,  1],
    [4, 16, 26, 16, 4],
    [7, 26, 41, 26, 7],
    [4, 16, 26, 16, 4],
    [1, 4,  7,  4,  1]
]
f3 = np.array(f2)
#f3 = 1/np.sum(f3) *np.array(f3)
filter.append(f3)

f4 = [
    [0, -1,  0],
    [-1, 4, -1],
    [0, -1,  0]
]
f4 = np.array(f4)
filter.append(f4)

f5 = [
    [-1, 0,  -1],
    [0,  4,   0],
    [-1, 0,  -1]
]
f5 = np.array(f5)
filter.append(f5)

f6 = [
    [1,   2,   1],
    [0,   0,   0],
    [-1, -2,  -1]
]
f6 = np.array(f6)
filter.append(f6)



## Homework Objective:
The objective of this homework is to explore the approximation of the
"adding process" within the convolution operation. This involves
defining functions for multi-adding (`MultiAdder`) and approximating
convolution (`MyConvAdder`). The degree of approximation can be
controlled by the variable `approx_until`. Additionally, we will compare
the results of approximate convolution with an exact convolution using
Python's own convolution function.

  

## Subtasks:

### 1.1 **Define the "MultiAdder" Function (5 points):**

-   Create a function called `MultiAdder` that takes a list/matrix of
    numbers and uses the `MyAdder` function to add them approximately.
-   The function should return the result of the multi-adding process.

In [ ]:

def MultiAdder(numbers, approx_until):
    array = np.array(numbers)
    sum = 0
    for element in np.nditer(array):
        sum = MyNbitAdder(sum,element, approx_until)
    return sum
    # sum = MyAdder(numbers, None)
MultiAdder([[1,2,3,4,5,6,7,8,9,10],[10,9,8,7,6,5,4,3,2,1]], 4)

128

### 1.2 Define the "MyConvAdder" Function (5 points):

-   Create a function called `MyConvAdder` that simulates the
    convolution operation using the `MultiAdder` function for the adding
    process.
-   The function should take an image and a 3x3 filter/kernel and return
    the convolved image.

In [9]:

def MyConvAdder(image, kernel, approx_until):
    image = np.array(image)
    kernel = np.array(kernel)
    image_shape = image.shape
    kernel_shape = kernel.shape
    result_array = np.zeros((image_shape[0]-kernel_shape[0]+1,image_shape[1]-kernel_shape[1]+1))
    for x in range(image_shape[0]-kernel_shape[0]+1):
        for y in range(image_shape[1]-kernel_shape[1]+1):
            result = np.multiply(np.flip(kernel),image[x:x+kernel_shape[0],y:y+kernel_shape[1]])
            result = MultiAdder(result,approx_until)
            result_array[x,y]=result
    # Your code here
    return result_array



### 1.3 Import the Sample Image and use the already defined filter(5 points):

-   Import the sample image and convert it to grayscale(already
    discussed in lecture). The sample image(man.jpeg) is also given in
    the resouces folder on Moodle.
-   Use a 3x3 gaussian filter/kernel (fltr) for convolution and apply
    the above `MyConvAdder` function on the sample image
-   Additionally, try out some of the given filters above f1-f6 and
    observe and document the effects on the convolved image



In [10]:
# Your code here
sample_image = np.array(io.imread("man.jpeg"))
sample_image = np.array((0.299 * np.array(sample_image[:,:,0])) + (0.587 * np.array(sample_image[:,:,1])) + (0.114 * np.array(sample_image[:,:,2])), dtype=int)

filter_kernel = fltr
result_images = []
for i in range(9):
    result_image = MyConvAdder(sample_image,filter_kernel,i)
    result_images.append(result_image)



### 1.4 Perform Approximate and Exact Convolution (5 points):

-   Use the `MyConvAdder` function to perform approximate convolution.
-   Use Python's own convolution function (e.g., from `scipy.signal`) to
    perform exact convolution.
-   Compare the results and calculate the error and similarity between
    the two convolution outputs.

In [47]:
# Your code here
from ipywidgets import interact, interactive, fixed, interact_manual, Output
import ipywidgets as widgets
from skimage.metrics import structural_similarity as ssim
from math import log10


def calcSimilarity(exact_result,approx_result):

    data_range=exact_result.max() - approx_result.max()  
    ssi = ssim(exact_result, approx_result, data_range=data_range)
    print(f"similarity {ssi}")
    return ssi

def calcError(exact_result,approx_result):
    max_error_distance = (exact_result-approx_result).max()  ### maximum error distance between two images
    print(f"Max Error Distance: {max_error_distance}")
    mse = np.mean((exact_result - approx_result)**2)         ### MSE should be closed to zreo
    print(f"Mean Squared Error: {mse}")
    psnr = 10 * log10((exact_result.max()**2) /mse)       ### PSNR should be closer to 54 dB
    print(f"Picture Signal to Noise Ratio {psnr}")
    return max_error_distance,mse,psnr

def process(sample_image, approx_until,filter_number):
    approx_convolution_result = MyConvAdder(sample_image, filter[filter_number], approx_until)
    exact_convolution_result = signal.convolve2d(sample_image,filter[filter_number],mode="valid")
    calcError(exact_convolution_result,approx_convolution_result)
    calcSimilarity(exact_convolution_result,approx_convolution_result)
    plt.subplot(1,2,1)
    plt.imshow(approx_convolution_result,cmap='gray')
    plt.title('Approx Convolution')

    plt.subplot(1,2,2)
    plt.imshow(exact_convolution_result,cmap='gray')
    plt.title('Exact Convolution')

    plt.show()

progress_output = Output()


w = interactive(process,sample_image=fixed(sample_image), 
                approx_until=widgets.IntSlider(min=0, max=20, step=1, value=4),
                filter_number=widgets.IntSlider(min=0, max=5, step=1, value=1))

display(w)




interactive(children=(IntSlider(value=4, description='approx_until', max=20), IntSlider(value=1, description='…

fig, ax = plt.subplots(9, figsize=(5,45))
for i in range(9):
    ax[i].imshow(exact_convolution_result-ressult[i], cmap="gray")
print(approx_convolution_result.shape)
print(exact_convolution_result.shape)